In [6]:
audio_original_file_path = "./meeting_audio.mp3"

In [31]:
from yt_dlp import YoutubeDL

# def download_video(url, save_path="."):
#     try:
#         yt = YouTube(url)

#         # Get the highest resolution stream available
#         stream = yt.streams.get_highest_resolution()

#         print(f"Downloading: {yt.title}")
#         stream.download(output_path=save_path)
#         print("Download complete!")

#     except Exception as e:
#         print(f"An error occurred: {e}")

def format_selector(ctx):
    """ Select the best video and the best audio that won't result in an mkv.
    NOTE: This is just an example and does not handle all cases """

    # formats are already sorted worst to best
    formats = ctx.get('formats')[::-1]

    # acodec='none' means there is no audio
    best_video = next(f for f in formats
                      if f['vcodec'] != 'none' and f['acodec'] == 'none')

    # find compatible audio extension
    audio_ext = {'mp4': 'm4a', 'webm': 'webm'}[best_video['ext']]
    # vcodec='none' means there is no video
    best_audio = next(f for f in formats if (
        f['acodec'] != 'none' and f['vcodec'] == 'none' and f['ext'] == audio_ext))

    # These are the minimum required fields for a merged format
    yield {
        'format_id': f'{best_video["format_id"]}+{best_audio["format_id"]}',
        'ext': best_video['ext'],
        'requested_formats': [best_video, best_audio],
        # Must be + separated list of protocols
        'protocol': f'{best_video["protocol"]}+{best_audio["protocol"]}'
    }

def download_video(url, save_path="."):
    URLS = [url]

    ydl_opts = {
        'format': format_selector,
        'output': save_path,
    }

    with YoutubeDL(ydl_opts) as ydl:
        ydl.download(URLS)

download_video("https://www.youtube.com/watch?v=-UEVMrSM8og", "./youtube_output.mp4")

[youtube] Extracting URL: https://www.youtube.com/watch?v=-UEVMrSM8og
[youtube] -UEVMrSM8og: Downloading webpage
[youtube] -UEVMrSM8og: Downloading tv client config
[youtube] -UEVMrSM8og: Downloading player 1080ef44
[youtube] -UEVMrSM8og: Downloading tv player API JSON
[youtube] -UEVMrSM8og: Downloading ios player API JSON
[youtube] -UEVMrSM8og: Downloading m3u8 information
[info] -UEVMrSM8og: Downloading 1 format(s): 616+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 1299
[download] Destination: Poor Boy Was Refused By A Girl, But Received SS-Rank Billionaire System - Mancap [-UEVMrSM8og].f616.mp4
[download] 100% of    2.61GiB in 00:25:01 at 1.78MiB/s                       
[download] Destination: Poor Boy Was Refused By A Girl, But Received SS-Rank Billionaire System - Mancap [-UEVMrSM8og].f140.m4a
[download] 100% of  112.55MiB in 00:00:54 at 2.07MiB/s     
[Merger] Merging formats into "Poor Boy Was Refused By A Girl, But Received SS-Rank Billionaire System -

In [1]:
import moviepy as mp

def extract_audio_with_context(video_path, output_audio_path):
    try:
        with mp.VideoFileClip(video_path) as video_clip:
            audio_clip = video_clip.audio
            audio_clip.write_audiofile(output_audio_path)
            print(f"Audio extracted and saved to: {output_audio_path}")
            audio_clip.close()

    except Exception as e:
        print(f"An error occurred: {e}")

video_file = "./meeting_record_video.mp4"
audio_file = "./meeting_audio.mp3"
extract_audio_with_context(video_file, audio_file)

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Google'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1920, 1080], 'bitrate': 1221, 'fps': 24.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 48000, 'bitrate': 128, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 2782.83, 'bitrate': 1353, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [1920, 1080], 'video_bitrate': 1221, 'video_fps': 24.0, 'default_audio_input_number': 0, 'def

MoviePy - Done.
Audio extracted and saved to: ./meeting_audio.mp3


In [4]:
import json

def save_object_to_json(obj, filename):
    try:
        with open(filename, 'w', encoding='utf-8') as f:  # Use utf-8 encoding
            json.dump(obj, f, indent=4, ensure_ascii=False) # Use indent for pretty printing, ensure_ascii for non-ascii support
        print(f"Object successfully saved to {filename}")
    except TypeError as e:
        print(f"Error: Object is not JSON serializable: {e}")
    except Exception as e:
        print(f"An error occurred while saving to {filename}: {e}")

def read_json_to_dict(filename) -> dict:
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            data = json.load(f)
            print(f"Data successfully loaded from {filename}")
            return data
    except FileNotFoundError:
        print(f"Error: File not found: {filename}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in {filename}")
        return None
    except Exception as e:
        print(f"An error occurred while reading {filename}: {e}")
        return None
    
def glossary_dict_to_str(d: dict) -> str:
    return "\n".join([f"{k} = {v}" for k, v in d.items()])

def glossary_list_to_str(l: list) -> str:
    return "\n".join(l)


In [30]:
import whisper

def transcribe_audio(audio_file_path):
    model = whisper.load_model("large")
    # audio = whisper.load_audio(audio_file_path)
    # mel = whisper.log_mel_spectrogram(audio).to(model.device)
    # options = whisper.DecodingOptions(language="th")
    result = model.transcribe(
        audio=audio_file_path,
        word_timestamps=True,
        language="th"
        )
    # result = whisper.decode(model, mel, options)
    return result

In [4]:
from pydub import AudioSegment

def split_audio_into_chunks(audio_original_file_path, output_folder, chunk_duration, overlap_duration) -> list[str]:
    output_file_paths = []
    song = AudioSegment.from_mp3(audio_original_file_path)
    chunk_length = chunk_duration * 1000
    overlap_length = overlap_duration * 1000
    start = 0
    while start < len(song):
        end = start + chunk_length
        chunk = song[start:end]
        output_file_path = f"{output_folder}/chunk_{len(output_file_paths)}.mp3"
        output_file_paths.append(output_file_path)
        chunk.export(output_file_path, format="mp3")
        start += chunk_length - overlap_length
    return output_file_paths

In [59]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def translate_text(text: str, glossary: dict) -> str:
    llm = ChatOpenAI(model="gpt-4o-mini")

    prompt = ChatPromptTemplate.from_messages([
        ("system",
        """
        You are narrative translator that can translate English into Thai.
        You will be provide with a long, unstructured block of information in English.
        Please rewrite it into clear, concise, and well-structured sentences.
        The primary goal is to make the text easy to understand when read aloud.
        Pay close attention to punctuation, sentence flow and make use of commas.
        Always use commas to separate clauses, phrases and sentence.
        Never use full stops to separate sentences.
        You will also provided glossary, so use them when necessary.
        Then translate the sentences into Thai, output only the Thai translation.

        Example input 1:
        Glossary:
        ```
        Valhalla = วัลฮาล่า
        ```
        English input:
        ```
        Story begins in a place called Valhalla
        one of the top three Hunter training
        institutions where countless Hunters
        apply but with an admission rate of 500
        to 1. not even 150 people can graduate
        with the name of Valhalla
        ```
        Example output 1:
        เรื่องราวเริ่มต้นขึ้นที่สถานที่ชื่อว่าวัลฮาล่า,
        ซึ่งเป็นหนึ่งในสามสถาบันฝึกอบรมฮันเตอร์ที่ดีที่สุด,
        มีฮันเตอร์จำนวนมากที่สมัครเข้าเรียน, แต่มีอัตราการรับเข้าเรียนที่น่าทึ่งถึง 500 ต่อ 1,
        ไม่ถึง 150 คนสามารถจบการศึกษาด้วยชื่อวัลฮาล่า,

        Example input 2:
        Glossary:
        ```
        Chen = เฉิน
        Shadong = ชาดง
        dog-licking = dog-licking
        ```
        English input:
        ```
        She pushes Chen in anger while going out of the store. 
        She also tells him that he has gone too far. 
        He looks at her, smirks, and thinks they still have a long way to go. 
        The system tells him to bind the dog-licking relationship no. 2 with Shadong.
        ```
        Example output 2:
        เธอผลักเฉินออกด้วยความโกรธขณะออกจากร้าน,
        และบอกเขาว่าเขาได้ทำเกินไป, 
        เขามองเธอ, ยิ้มเยาะ, และคิดว่าหนทางยังมีอีกยาวไกล,
        ระบบบอกให้เขา bind ความสัมพันธ์ dog-licking ลำดับที่สองกับชาดง,
        """),
        ("user",
        """
        Glossary:
        ```
        {glossary}
        ```
        English input:
        ```
        {sentences}
        ```
        """),
    ])

    output_parser = StrOutputParser()

    chain = prompt | llm | output_parser
    output = chain.invoke({"sentences": text, "glossary": glossary_dict_to_str(glossary)})
    return output

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def enhance_transcribed_meeting(text: str, glossary: list) -> str:
    llm = ChatOpenAI(model="gpt-4o-mini")

    prompt = ChatPromptTemplate.from_messages([
        ("system",
        """
        You are a specialized meeting transcription editor for software development industry.
        You will be given a meeting transcript.
        The meeting transcript will be in Thai and technical term will be in English.
        Your task is to correct any mis-transcribed words, especially those related to software development.
        Consider the surrounding words and sentences to understand the intended meaning.
        If a word is unclear, try to infer the most likely correct word based on context.
        If multiple corrections are possible, choose the one that best fits the overall topic of the meeting.
        Use your knowledge of industry to identify the most likely correct words.
        You will also be provided with a glossary of technical terms, use them when necessary.
        Return only the corrected transcript.
        """),
        ("user",
        """
        Glossary:
        ```
        {glossary}
        ```
        Original meeting transcript:
        ```
        {sentences}
        ```
        """),
    ])

    output_parser = StrOutputParser()

    chain = prompt | llm | output_parser
    output = chain.invoke({"sentences": text, "glossary": glossary_list_to_str(glossary)})
    return output

In [ ]:
def generate_speech_from_text(text, output_file_path):
  import azure.cognitiveservices.speech as speechsdk

  # Creates an instance of a speech config with specified subscription key and service region.
  speech_key = ""
  service_region = "southeastasia"

  speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
  # Note: the voice setting will not overwrite the voice element in input SSML.
  # speech_config.speech_synthesis_voice_name = "th-TH-AcharaNeural"
  speech_config.speech_synthesis_voice_name = "th-TH-NiwatNeural"
  audio_config = speechsdk.audio.AudioOutputConfig(filename=output_file_path)

  # use the default speaker as audio output.
  speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)

  result = speech_synthesizer.speak_text_async(text).get()
  # Check result
  if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
      print("Speech synthesized for text [{}]".format(text))
  elif result.reason == speechsdk.ResultReason.Canceled:
      cancellation_details = result.cancellation_details
      print("Speech synthesis canceled: {}".format(cancellation_details.reason))
      if cancellation_details.reason == speechsdk.CancellationReason.Error:
          print("Error details: {}".format(cancellation_details.error_details))



In [9]:
import os

In [61]:
import os
def process_audio(input_audio_file_path, glossary):
    print(f"Transcribing {os.path.basename(input_audio_file_path)}")
    result = transcribe_audio(input_audio_file_path)
    # get file name without extension
    input_audio_file_path_without_ext = os.path.basename(input_audio_file_path).split(".")[0]
    translated_result = translate_text(result['text'], glossary)
    result['translated_text'] = translated_result
    save_object_to_json(result, f"{input_audio_file_path_without_ext}.json")
    generate_speech_from_text(translated_result, f"{input_audio_file_path_without_ext}_translated.mp3")

In [7]:
output_file_paths = split_audio_into_chunks(audio_original_file_path, ".", 8*60, 10)

In [31]:
for output_file_path in output_file_paths:
    result = transcribe_audio(output_file_path)
    input_audio_file_path_without_ext = os.path.basename(output_file_path).split(".")[0]
    save_object_to_json(result, f"{input_audio_file_path_without_ext}.json")
    break

/Users/kaewsai/miniconda3/envs/translate-sub/lib/python3.12/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, 

Object successfully saved to chunk_0.json


In [11]:
glossary = read_json_to_dict("nocnoc_glossary.json")["terms"]
info = read_json_to_dict("chunk_0.json")
enhanced_result = enhance_transcribed_meeting(info['text'], glossary)
print(enhanced_result)

Data successfully loaded from nocnoc_glossary.json
Data successfully loaded from chunk_0.json
```
ครับ ตอนนี้ขอคอนเฟิร์มหน่อยนะครับว่าตอนนี้สิ่งที่น้อง ๆ คล้อยมีที่เป็น Tool ที่เรามีครับตอนนี้มันจะมีตัวที่มันไปเอาของจาก S3 มาสร้างเป็น Request แล้วก็ปั้น API ยิงไปหา NetSuite ใช่ป่ะครับแล้วก็เสร็จปุ๊บก็ Save ข้อมูลที่ได้ลง Database ครับผม ใช่ครับมีเท่านี้เลยใช่ไหมครับ ใช่ครับโอเค งั้นฝากนายช่วย Go To ให้หน่อยได้ไหมครับว่าแต่ละตัวเนี่ยมันใช้เทคโนโลยีอะไรบ้างแล้วก็มัน Implement มี Repo อะไรตรงไหนบ้างโอเคครับโอเคครับโอเค อันนี้นะ ขอขึ้นเชิญครับครับโอเคครับ ก็ของตัว NocNoc Choice เดี๋ยวผมมาให้ดูดีด้วยก็มันจะมีของ NocNoc Choice ตัวเนี่ย มันจะมีข้อมูลที่ตั้งต้นมาจาก S3 มันจะมีผมเรียกไป 2 Source แล้วกันก็คือ Source 1 มันจะมาจากตัว Toll เนี่ยตัว Toll นี้จะเป็น เหมือนกับเขาทำ FTP มาให้มาวางไว้ครับก็จะเป็นตัว Folder ที่เป็น Toll B2C ตรงนี้ครับครับตัวนี้ก็จะเป็น เหมือน Source ตั้งต้นที่ผมจะไปดึงนะ เอาแต่ละรายการมาของเขาทิ้ง เขาไปปั้น API ที่ส่งไปหาตัว Made in Sweden แล้วก็ครับแล้วก็จะมีของที่อื่น ๆ

In [10]:
input_audio_file_path_without_ext = os.path.basename(output_file_path).split(".")[0]
save_object_to_json(result, f"{input_audio_file_path_without_ext}.json")

Object successfully saved to chunk_0.json


In [57]:
glossary = read_json_to_dict("glossary.json")
for output_file_path in output_file_paths:
    process_audio(output_file_path, glossary)
    break

Data successfully loaded from glossary.json
Transcribing chunk_0.mp3


/Users/kaewsai/miniconda3/envs/translate-sub/lib/python3.12/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, 

Object successfully saved to chunk_0.json
Speech synthesized for text [เรื่องราวนี้เริ่มต้นในห้องของอาคารที่พัก, เฉินหยวนกำลังมองไปที่หน้าจอโทรศัพท์และหลั่งน้ำตา, 
สาเหตุที่ทำให้เขาร้องไห้คือข้อความที่แฟนของเขาส่งมา, ซึ่งเธอกำลังจะเลิกกับเขา. 
เขาถามเธอว่าทำไมอยู่ ๆ ถึงเป็นแบบนี้, ขณะร้องไห้เขาจึงส่งข้อความไปหาเธอและสัญญาว่าจะทำงานหนักเพื่อให้เธอมีชีวิตที่มีความสุข, 
แต่เธอกลับบล็อกเขาก่อนที่เขาจะส่งข้อความ. 
เขานอนข้ามเก้าอี้เหมือนคนตาย, เพื่อนร่วมห้องไม่ทราบว่าเกิดอะไรขึ้นกับเขา และเรียกเขาให้ไปเล่นเกม. 
แต่เขากลับไม่สนใจและวิ่งออกจากห้อง, เมื่อเห็นพฤติกรรมแปลก ๆ ของเขา, พวกเขาคิดว่ามันคือการเลิกราอีกครั้ง, 
นี่คือครั้งที่หกในเดือนนี้ที่เขาต้องเผชิญกับการเลิกรา. 
บนถนน, ผู้คนมองเขาและเริ่มกระซิบเกี่ยวกับเขา, พวกเขารู้จักเฉินหยวนในฐานะ dog licker ที่มีชื่อเสียง. 
เฉินมาที่นี่เพื่อตอบตกลงกับผู้หญิงที่เพิ่งเลิกกับเขา, ชื่อของเธอคือชาดง. 
เธอบอกเขาไม่ให้แสดงพฤติกรรมแบบนี้ในที่สาธารณะเพราะทุกคนกำลังมองพวกเขา. 
เขาบอกเธอว่าเขารู้ว่าเธอโกรธเขา, ดังนั้นเขาจึงนำหมูบัวเก๊า ซึ่งเป็นอาหารที่เธอช

In [63]:
filename = "chunk_0.json"
output_file_path_without_ext = os.path.basename(filename).split(".")[0]
glossary = read_json_to_dict("glossary.json")
info = read_json_to_dict(filename)
translated_result = translate_text(info['text'], glossary)
info['translated_text'] = translated_result
save_object_to_json(info, f"{output_file_path_without_ext}.json")
generate_speech_from_text(translated_result, f"{output_file_path_without_ext}_translated.mp3")

Data successfully loaded from glossary.json
Data successfully loaded from chunk_0.json
Object successfully saved to chunk_0.json
Speech synthesized for text [เรื่องราวเริ่มต้นในห้องในอาคารที่พัก, ที่เฉินหยวนกำลังมองหน้าจอโทรศัพท์และร้องไห้, สาเหตุที่เขาร้องไห้คือข้อความที่แฟนสาวของเขาส่งมา, เธอกำลังบอกเลิกกับเขา, เขาถามเธอว่าทำไมถึงเป็นแบบนี้อย่างกะทันหัน, ขณะร้องไห้ เขาส่งข้อความไปหเธอและสัญญาว่าจะพยายามทำงานหนักเพื่อให้เธอมีชีวิตที่มีความสุข, แต่เธอก็บล็อกเขาก่อนที่เขาจะส่งข้อความ, เขานอนอยู่บนเก้าอี้เหมือนคนตาย, เพื่อนร่วมห้องของเขาไม่รู้ว่าเกิดอะไรขึ้นกับเขาและเรียกเขาไปเล่นเกม, แต่เขาก็เมินพวกเขาและวิ่งออกจากห้อง, เมื่อเห็นพฤติกรรมแปลก ๆ ของเขา, พวกเขาคิดว่ามันเป็นอีกกรณีของการเลิกกัน, นี่คือครั้งที่หกในเดือนนี้ที่เขาเผชิญกับการเลิกกัน, บนถนนผู้คนมองเขาและเริ่มกระซิบเกี่ยวกับเขา, พวกเขารู้เฉินหยวนว่าเป็นที่รู้จักในฐานะ dog licker ชื่อดัง, เฉินมาที่นี่เพื่อพบกับหญิงสาวที่เพิ่งบอกเลิกกับเขา, เธอชื่อชาดง, เธอบอกเขาว่าอย่าทำตัวแบบนี้ต่อหน้าสาธารณะเพราะทุกคนกำลังมองพวกเขา, เขาบอกเธอว่า

In [44]:
glossary = read_json_to_dict("glossary.json")

# turn dict to string
def dict_to_str(d: dict) -> str:
    return "\n".join([f"{k} = {v}" for k, v in d.items()])
glossary_str = dict_to_str(glossary)
print(glossary_str)

Data successfully loaded from glossary.json
Chen = เฉิน
Shadong = ชาตง
dog-licking = dog-licking
